In [0]:
import pandas as pd
from google.colab import files
import io

import math
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
#from tf_utils import load_dataset, random_mini_batches, convert_to_one_hot, predict

%matplotlib inline

In [0]:
uploaded = files.upload()

Saving color-test-data.csv to color-test-data.csv


In [0]:
#color_data_df = pd.read_csv(io.StringIO(uploaded["color-data-train.csv"].decode("utf-8")))
color_test_df = pd.read_csv(io.StringIO(uploaded["color-test-data.csv"].decode("utf-8")))

In [0]:
#Loading dataset from color-data-train.csv. X_train has 4 features red,green,blue,text-color(0 for black,1 for white). Y_train has labels 0("Can't See) or 1("Can See")."
Y_train = np.array(color_data_df["0.4"])
Y_train = np.float32(Y_train.T)
Y_train = Y_train.reshape(1,Y_train.shape[0])             #shape of Y_train = (1,37587)
X_train = np.array(color_data_df.iloc[:,:4])
X_train = np.float32(X_train.T)                           #shape of X_train = (4,37587)

#Normalizing features. Max value for red, green, blue features is 256, so divide each feature value by 256. For "text-color" feature only possible values are 0 or 1. So don't normalize this feature.
X_train[:3,:] = X_train[:3,:]/256

#Same for train set
Y_test = np.array(color_test_df["0.4"])
Y_test = np.float32(Y_test.T)
Y_test = Y_test.reshape(1,Y_test.shape[0])
X_test = np.array(color_test_df.iloc[:,:4])
X_test = np.float32(X_test.T)


In [0]:
def create_placeholders(n_x, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_x -- scalar, size of an image vector (num_px * num_px = 64 * 64 * 3 = 12288)
    n_y -- scalar, number of classes (from 0 to 5, so -> 6)
    
    Returns:
    X -- placeholder for the data input, of shape [n_x, None] and dtype "float"
    Y -- placeholder for the input labels, of shape [n_y, None] and dtype "float"
    """

    X = tf.placeholder("float32", [n_x, None])
    Y = tf.placeholder("float32", [n_y, None])
    
    return X, Y

In [0]:
def initialize_parameters():
    """
    Initializes parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [3, 4]  3 neurons in the first layer, 4 input features
                        b1 : [3, 1]
                        W2 : [1, 3]
                        b2 : [1, 1]
    
    Returns:
    parameters -- a dictionary of tensors containing W1, b1, W2, b2
    """
        
    W1 = tf.get_variable("W1", [3,4], initializer = tf.contrib.layers.xavier_initializer(), dtype="float32")
    b1 = tf.get_variable("b1", [3,1], initializer = tf.zeros_initializer(), dtype="float32")
    W2 = tf.get_variable("W2", [1,3], initializer = tf.contrib.layers.xavier_initializer(), dtype="float32")
    b2 = tf.get_variable("b2", [1,1], initializer = tf.zeros_initializer(), dtype="float32")
    
  
    parameters = {"W1": W1,
                  "b1": b1,
                  "W2": W2,
                  "b2": b2}
    '''
    
    parameters = {'W1': tf.get_variable("W1", initializer = tf.convert_to_tensor(np.array([[ 0.31035033, -0.7829059 , -0.41219217, -0.16607714],
                [-1.3012043 , -3.715908  , -0.24158937,  5.1152887 ],
                [  0.42639863,  4.409025  ,  0.26884907,  0.55255806  ]],
                dtype="float32"))),
                'W2': tf.get_variable("W2", initializer = tf.convert_to_tensor(np.array([[ -0.7082019, 110.77128  , -27.845854 ]], dtype="float32"))),
                'b1': tf.get_variable("b1", initializer = tf.convert_to_tensor(np.array([[-2.8216890e-01],
                [-4.620715e-03],
                [-5.049001e+00]], dtype="float32"))),
                'b2': tf.get_variable("b2", initializer = tf.convert_to_tensor(np.array([[0.98699754]], dtype="float32")))}
    '''
    return parameters

In [0]:
def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model: LINEAR -> RELU -> LINEAR -> RELU -> LINEAR -> SOFTMAX
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "b1", "W2", "b2", "W3", "b3"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    b1 = parameters['b1']
    W2 = parameters['W2']
    b2 = parameters['b2']
   
    Z1 = tf.add(tf.matmul(W1,X), b1)                                              # Z1 = np.dot(W1, X) + b1
    A1 = tf.nn.relu(Z1)                                              # A1 = relu(Z1)
    Z2 = tf.add(tf.matmul(W2,A1), b2)                                              # Z2 = np.dot(W2, a1) + b2
    #A2 = tf.nn.sigmoid(Z2)                                           #final layer use sigmoid as only 2 classes "Can't See" or "Can See"
    
    return Z2

In [0]:
def compute_cost(AL, Y, params):
    """
    Implement the cost function defined by equation (7).

    Arguments:
    AL -- probability vector corresponding to your label predictions, shape (1, number of examples)
    Y -- true "label" vector (for example: containing 0 if non-cat, 1 if cat), shape (1, number of examples)

    Returns:
    cost -- cross-entropy cost
    """
    labels = tf.transpose(Y)
    logits = tf.transpose(AL)
    beta = 0.01
    #m = Y.shape[1]
    # Compute loss from aL and y.
    cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logits, labels = labels))
    #cost = tf.divide(tf.reduce_sum(tf.add(tf.matmul(Y,tf.log(tf.transpose(AL))), tf.matmul(tf.subtract(1.0,Y),tf.log(tf.subtract(1.0,tf.transpose(AL))))), tf.multiply(-1.0, m)))
   
    cost = tf.squeeze(cost)      # To make sure your cost's shape is what we expect (e.g. this turns [[17]] into 17).
    
    regularizers = tf.squeeze(tf.nn.l2_loss(tf.squeeze(params["W1"])) + tf.nn.l2_loss(tf.squeeze(params["W2"])))
    cost = tf.squeeze(tf.reduce_mean(cost + beta * regularizers))
    return cost

In [0]:
def random_mini_batches(X, Y, mini_batch_size = 64):
    """
    Creates a list of random minibatches from (X, Y)
    
    Arguments:
    X -- input data, of shape (input size, number of examples)
    Y -- true "label" vector (1 for blue dot / 0 for red dot), of shape (1, number of examples)
    mini_batch_size -- size of the mini-batches, integer
    
    Returns:
    mini_batches -- list of synchronous (mini_batch_X, mini_batch_Y)
    """
    
    m = X.shape[1]                  # number of training examples
    mini_batches = []
        
    # Step 1: Shuffle (X, Y)
    permutation = list(np.random.permutation(m))
    shuffled_X = X[:, permutation]
    shuffled_Y = Y[:, permutation].reshape((1,m))

    # Step 2: Partition (shuffled_X, shuffled_Y). Minus the end case.
    num_complete_minibatches = math.floor(m/mini_batch_size) # number of mini batches of size mini_batch_size in your partitionning
    for k in range(0, num_complete_minibatches):
        mini_batch_X = shuffled_X[:, k*mini_batch_size : (k*mini_batch_size) + mini_batch_size]
        mini_batch_Y = shuffled_Y[:, k*mini_batch_size : (k*mini_batch_size) + mini_batch_size]

        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    # Handling the end case (last mini-batch < mini_batch_size)
    if m % mini_batch_size != 0:
        filled_egs = num_complete_minibatches * mini_batch_size
        mini_batch_X = shuffled_X[:, filled_egs :]
        mini_batch_Y = shuffled_Y[:, filled_egs :]
      
        mini_batch = (mini_batch_X, mini_batch_Y)
        mini_batches.append(mini_batch)
    
    return mini_batches

In [0]:
def model(X_train, Y_train, X_test, Y_test, learning_rate = 0.01, num_epochs = 1000, minibatch_size = 20, print_cost = True): 
    """
    starting LR = 0.0001
    lambda values 0.01, 0.02, ....
    Implements a three-layer tensorflow neural network: LINEAR->RELU->LINEAR->RELU->LINEAR->SOFTMAX.
    
    Arguments:
    X_train -- training set, of shape (input size = 4, number of training examples = 37587)
    Y_train -- test set, of shape (output size = 6, number of training examples = 1080)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    tf.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    (n_x, m) = X_train.shape                          # (n_x: input size, m : number of examples in the train set)
    n_y = Y_train.shape[0]                            # n_y : output size
    costs = []                                        # To keep track of the cost
    
  
    # Create Placeholders of shape (n_x, n_y)
    X, Y = create_placeholders(n_x, n_y)

    # Initialize parameters
    parameters = initialize_parameters()
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    Z3 = forward_propagation(X, parameters)
    
    # Cost function: Add cost function to tensorflow graph
    cost = compute_cost(Z3, Y, parameters)
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer.
    optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(cost)
    
    # Initialize all the variables
    init = tf.global_variables_initializer()

    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            epoch_cost = 0.                       # Defines a cost related to an epoch
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the "optimizer" and the "cost", the feedict should contain a minibatch for (X,Y).
                _ , minibatch_cost = sess.run([optimizer, cost], feed_dict={X: minibatch_X, Y: minibatch_Y})
               
                
                epoch_cost += minibatch_cost / num_minibatches
             
           
            # Print the cost every epoch
            if print_cost == True and epoch % 100 == 0:
                print ("Cost after epoch %i: %f" % (epoch, epoch_cost))
                #parameters = sess.run(parameters)
                Z3_sigmoid = tf.nn.sigmoid(Z3)
                prediction = np.where(np.array(Z3_sigmoid.eval({X:X_train})) <= 0.5, 0.0, 1.0)
                correct_prediction = tf.equal(np.float32(prediction), Y)
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
                print ("Train Accuracy:", accuracy.eval({X:X_train, Y:Y_train}))
                
                
                prediction = np.where(np.array(Z3_sigmoid.eval({X:X_test})) <= 0.5, 0.0, 1.0)
                correct_prediction = tf.equal(np.float32(prediction), Y)
                accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
                print("Test Accuray:", accuracy.eval({X:X_test, Y:Y_test}))
            if print_cost == True and epoch % 5 == 0:
                costs.append(epoch_cost)
            

                
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()
        '''
        # lets save the parameters in a variable
        parameters = sess.run(parameters)
        print ("Parameters have been trained!")

        # Calculate the correct predictions
        correct_prediction = tf.equal(tf.argmax(Z3), tf.argmax(Y))

        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

        print ("Train Accuracy:", accuracy.eval({X: X_train, Y: Y_train}))
        '''
        return parameters

In [0]:
params = model(X_train, Y_train, X_test, Y_test)


Cost after epoch 0: 0.505432
Train Accuracy: 0.8525554
Test Accuray: 0.6204805
Cost after epoch 100: 0.005210
Train Accuracy: 0.99874955
Test Accuray: 0.5000133


In [0]:
print(params)


{'W1': array([[-1.936802  , -6.6421294 , -0.41709325,  8.621973  ],
       [ 6.0473714 , 11.235168  ,  3.1842556 ,  0.6006382 ],
       [ 6.116137  , 12.880785  ,  3.72343   , -1.4074022 ]],
      dtype=float32), 'b1': array([[ 0.03547624],
       [-5.787856  ],
       [-0.23749885]], dtype=float32), 'W2': array([[ 65.09368 , -11.056453,   7.79873 ]], dtype=float32), 'b2': array([[-8.094122]], dtype=float32)}


In [0]:
params = {'W1': np.array([[ 1.7197353,  5.162647 ,  0.7523546, -7.434905 ],
       [ 1.8272332,  5.5076814,  0.8147408, -7.9281483],
       [ 1.7512232,  3.253422 ,  0.3704751, -2.0858917]], dtype="float32"), 'b1': np.array([[-0.20724566],
       [-0.2199342 ],
       [-2.8797793 ]], dtype="float32"), 'W2': np.array([[ 23.942451,  24.787357, -34.443848]], dtype="float32"), 'b2': np.array([[0.98434925]], dtype="float32")}


In [0]:
Z3 = forward_propagation(X_test,params)
Z3_sigmoid = tf.nn.sigmoid(Z3)

with tf.Session() as sess:
  prediction = np.where(np.array(sess.run(Z3_sigmoid)) <= 0.5, 0, 1)
  correct_prediction = tf.equal(prediction, Y_test)
  accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  print ("Train Accuracy:", accuracy.eval())
  

Train Accuracy: 0.49929497
